In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split  # Import train_test_split function
from tensorflow.keras.applications import VGG16


2024-02-26 16:42:27.867893: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 16:42:27.868014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 16:42:27.980046: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:

# Function to extract frames from video
def extract_frames(video_path, target_height, target_width, num_frames):
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, num=num_frames, dtype=np.int)

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (target_width, target_height))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)

    cap.release()
    return frames




In [3]:
# Function to preprocess video data
def preprocess_data(data_dir, target_height, target_width, num_frames):
    X = []
    y = []
    for video_name in os.listdir(data_dir):
        if video_name.startswith('fi'):  # Violence video
            label = 1
        elif video_name.startswith('no'):  # Non-violence video
            label = 0
        else:
            continue

        video_path = os.path.join(data_dir, video_name)
        frames = extract_frames(video_path, target_height, target_width, num_frames)
        if len(frames) == num_frames:
            X.append(frames)
            y.append(label)

    X = np.array(X)
    y = np.array(y)
    return X, y


In [4]:
# Define CNN-LSTM model architecture
def create_model(input_shape):
    model = models.Sequential()
    model.add(layers.TimeDistributed(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'), input_shape=input_shape))
    model.add(layers.TimeDistributed(layers.MaxPooling2D(pool_size=(2, 2))))
    model.add(layers.TimeDistributed(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')))
    model.add(layers.TimeDistributed(layers.MaxPooling2D(pool_size=(2, 2))))
    model.add(layers.TimeDistributed(layers.Flatten()))
    model.add(layers.LSTM(64))
    model.add(layers.Dense(1, activation='sigmoid'))

    return model
